In [1]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token:str, **kwargs) -> None:
        print(f"Received streaming token {token}, kwargs:{kwargs}")
        
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Received non-streaming response: {response}")

In [2]:
from llmstudio.llm.langchain import ChatLLMstudio

Running LLMstudio Engine on http://localhost:50001 Running LLMstudio Tracking on http://localhost:50002 



In [3]:
gpt3 = ChatLLMstudio(model_id='openai/gpt-3.5-turbo', temperature=0)

In [8]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

@tool
def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."


def assistant(question: str)->str:
    """YOUR CODE HERE"""
    tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, gpt3, agent=AgentType.OPENAI_MULTI_FUNCTIONS, handle_parsing_errors="Check your output and make sure it conforms!", verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [9]:
assistant('When does my train depart? My ticket number is 1234')




> Entering new AgentExecutor chain...


/Users/claudiolemos/miniconda3/envs/playground/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


INFO:     ::1:53987 - "POST /api/tracking/logs HTTP/1.1" 200 OK
INFO:     ::1:53984 - "POST /api/engine/chat/openai HTTP/1.1" 200 OK

Invoking: `get_departure` with `{'ticket_number': '1234'}`


12:00 AMINFO:     ::1:53987 - "POST /api/tracking/logs HTTP/1.1" 200 OK
INFO:     ::1:53989 - "POST /api/engine/chat/openai HTTP/1.1" 200 OK
Your train departs at 12:00 AM.

> Finished chain.


{'input': 'When does my train depart? My ticket number is 1234',
 'output': 'Your train departs at 12:00 AM.'}